# sustainability_eda

In [1]:
pwd

'/Users/kelly/metis_v3/Project_4'

In [88]:
import numpy as np
import pandas as pd

In [2]:
from pymongo import MongoClient
import json
import pprint
import re
import string

In [3]:
import nltk

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import wordpunct_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

In [63]:
from textblob import TextBlob

In [112]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.naive_bayes import MultinomialNB

In [4]:
client = MongoClient()
db = client.environment
sustainability_collection = db.sustainability

In [5]:
print("Number of documents collected:", sustainability_collection.count())

Number of documents collected: 10379


In [6]:
#Use the first Tweet as an example

sustainability_collection.find_one()

{'_id': ObjectId('5b70e2403386b77533324b26'),
 'created_at': 'Mon Aug 13 01:41:17 +0000 2018',
 'id': 1028818744703950849,
 'id_str': '1028818744703950849',
 'text': 'RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'angelurena',
    'name': 'Angel Ureña',
    'id': 70707457,
    'id_str': '70707457',
    'indices': [3, 14]},
   {'screen_name': 'BillClinton',
    'name': 'Bill Clinton',
    'id': 1330457336,
    'id_str': '1330457336',
    'indices': [95, 107]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_nam

In [7]:
print("User ID:  ", '\t',sustainability_collection.find_one()['_id'], '\nUserLocation:   ',sustainability_collection.find_one()['user']['location'], '\nTweet:  ','\t',sustainability_collection.find_one()['text'])

User ID:   	 5b70e2403386b77533324b26 
UserLocation:    Texas 
Tweet:   	 RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…


In [8]:
#First 5 Tweets

for post in sustainability_collection.find().limit(5):
    print(pprint.pprint(post['user']['location']),'\n',(pprint.pprint(post['text'])))

'Texas'
('RT @angelurena: This visit concludes this week’s stops in Miami and the '
 'Caribbean by President @BillClinton to promote sustainability and r…')
None 
 None
''
('Are you in Melbourne or Sydney? Join us at #SPFAustralia to learn about the '
 'latest trends in #EHS, #RiskManagement… https://t.co/BG7eWM1FCO')
None 
 None
'Sydney, New South Wales'
('Why Vienna Does So Well on Quality-of-Life Rankings #affordablehousing '
 '#publictransport #sustainability https://t.co/xLQASUUqoc')
None 
 None
''
('RT @APLShipping: APL achieved a 50.7% reduction in CO2 emissions per '
 'transported container per kilometre in 2017, as compared to 2009. Read…')
None 
 None
''
("RT @AmazonWatch: “Hydroelectricity may appear relatively 'clean' but the "
 'process in which a mega-dam is built is far from it... #BeloMonte…')
None 
 None


In [39]:
my_tweet = post['text']

In [40]:
print(my_tweet)

RT @AmazonWatch: “Hydroelectricity may appear relatively 'clean' but the process in which a mega-dam is built is far from it... #BeloMonte…


**Tokenize Example Tweet**

In [49]:
print(word_tokenize(my_tweet))

['RT', '@', 'AmazonWatch', ':', '“', 'Hydroelectricity', 'may', 'appear', 'relatively', "'clean", "'", 'but', 'the', 'process', 'in', 'which', 'a', 'mega-dam', 'is', 'built', 'is', 'far', 'from', 'it', '...', '#', 'BeloMonte…']


In [50]:
#simple cleaning function

def cleaned_text(my_text):
    
    words = re.sub('[%s]'% re.escape(string.punctuation), ' ', post['text']) 
    words = list(word_tokenize(words))
    words = [re.sub('[%s]'% re.escape(string.punctuation), ' ', word) for word in words]
    words=[word.lower() for word in words if word.isalpha()]
    
    return words

In [51]:
cleaned_text(my_tweet)

['rt',
 'amazonwatch',
 'hydroelectricity',
 'may',
 'appear',
 'relatively',
 'clean',
 'but',
 'the',
 'process',
 'in',
 'which',
 'a',
 'mega',
 'dam',
 'is',
 'built',
 'is',
 'far',
 'from',
 'it']

In [52]:
words = cleaned_text(my_tweet)

In [53]:
#Bi-grams

twograms = list(ngrams(words,2))
for row in twograms:
    print(row[0],row[1])

rt amazonwatch
amazonwatch hydroelectricity
hydroelectricity may
may appear
appear relatively
relatively clean
clean but
but the
the process
process in
in which
which a
a mega
mega dam
dam is
is built
built is
is far
far from
from it


Try Other Ways of Tokenizing

In [54]:
sentences = sent_tokenize(my_tweet)
print(sentences)

["RT @AmazonWatch: “Hydroelectricity may appear relatively 'clean' but the process in which a mega-dam is built is far from it... #BeloMonte…"]


In [55]:
tokenizer = TreebankWordTokenizer()
tokenizer.tokenize(sentences[0])

['RT',
 '@',
 'AmazonWatch',
 ':',
 '“',
 'Hydroelectricity',
 'may',
 'appear',
 'relatively',
 "'clean",
 "'",
 'but',
 'the',
 'process',
 'in',
 'which',
 'a',
 'mega-dam',
 'is',
 'built',
 'is',
 'far',
 'from',
 'it',
 '...',
 '#',
 'BeloMonte…']

In [56]:
wordpunct_tokenize(sentences[0])

['RT',
 '@',
 'AmazonWatch',
 ':',
 '“',
 'Hydroelectricity',
 'may',
 'appear',
 'relatively',
 "'",
 'clean',
 "'",
 'but',
 'the',
 'process',
 'in',
 'which',
 'a',
 'mega',
 '-',
 'dam',
 'is',
 'built',
 'is',
 'far',
 'from',
 'it',
 '...',
 '#',
 'BeloMonte',
 '…']

**Parts of Speech Tagging**

In [57]:
'''
WP: wh-pronoun ("who", "what")
VBZ: verb, 3rd person sing. present ("takes")
VBG: verb, gerund/present participle ("taking")
TO: to ("to go", "to him")
DT: determiner ("the", "this")
NN: noun, singular or mass ("door")
.: Punctuation (".", "?")
'''

'\nWP: wh-pronoun ("who", "what")\nVBZ: verb, 3rd person sing. present ("takes")\nVBG: verb, gerund/present participle ("taking")\nTO: to ("to go", "to him")\nDT: determiner ("the", "this")\nNN: noun, singular or mass ("door")\n.: Punctuation (".", "?")\n'

In [58]:
for word, pos in pos_tag(words):
    print("Word:", word, '\t\t', "Part of Speech:", pos)


Word: rt 		 Part of Speech: NN
Word: amazonwatch 		 Part of Speech: NN
Word: hydroelectricity 		 Part of Speech: NN
Word: may 		 Part of Speech: MD
Word: appear 		 Part of Speech: VB
Word: relatively 		 Part of Speech: RB
Word: clean 		 Part of Speech: JJ
Word: but 		 Part of Speech: CC
Word: the 		 Part of Speech: DT
Word: process 		 Part of Speech: NN
Word: in 		 Part of Speech: IN
Word: which 		 Part of Speech: WDT
Word: a 		 Part of Speech: DT
Word: mega 		 Part of Speech: NN
Word: dam 		 Part of Speech: NN
Word: is 		 Part of Speech: VBZ
Word: built 		 Part of Speech: VBN
Word: is 		 Part of Speech: VBZ
Word: far 		 Part of Speech: RB
Word: from 		 Part of Speech: IN
Word: it 		 Part of Speech: PRP


**Chunking**

In [59]:
tags = pos_tag(words)
tree = ne_chunk(tags)
print(tags)
print(tree)

[('rt', 'NN'), ('amazonwatch', 'NN'), ('hydroelectricity', 'NN'), ('may', 'MD'), ('appear', 'VB'), ('relatively', 'RB'), ('clean', 'JJ'), ('but', 'CC'), ('the', 'DT'), ('process', 'NN'), ('in', 'IN'), ('which', 'WDT'), ('a', 'DT'), ('mega', 'NN'), ('dam', 'NN'), ('is', 'VBZ'), ('built', 'VBN'), ('is', 'VBZ'), ('far', 'RB'), ('from', 'IN'), ('it', 'PRP')]
(S
  rt/NN
  amazonwatch/NN
  hydroelectricity/NN
  may/MD
  appear/VB
  relatively/RB
  clean/JJ
  but/CC
  the/DT
  process/NN
  in/IN
  which/WDT
  a/DT
  mega/NN
  dam/NN
  is/VBZ
  built/VBN
  is/VBZ
  far/RB
  from/IN
  it/PRP)


In [60]:
tree.draw()

**Print the Stem of Each Word**

In [69]:
stemmer = nltk.stem.porter.PorterStemmer()

for word in words:
    print(stemmer.stem(word))

rt
amazonwatch
hydroelectr
may
appear
rel
clean
but
the
process
in
which
a
mega
dam
is
built
is
far
from
it


**TextBlob Sentiment Analysis**

In [65]:
TextBlob(my_tweet).sentiment

Sentiment(polarity=0.23333333333333334, subjectivity=0.8500000000000001)

In [73]:
words

['rt',
 'amazonwatch',
 'hydroelectricity',
 'may',
 'appear',
 'relatively',
 'clean',
 'but',
 'the',
 'process',
 'in',
 'which',
 'a',
 'mega',
 'dam',
 'is',
 'built',
 'is',
 'far',
 'from',
 'it']

In [82]:
#Put first five tweets into a list to analyze

five_tweets = []
    
for post in sustainability_collection.find().limit(5): 
    five_tweets.append(post['text'])

In [83]:
five_tweets

['RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…',
 'Are you in Melbourne or Sydney? Join us at #SPFAustralia to learn about the latest trends in #EHS, #RiskManagement… https://t.co/BG7eWM1FCO',
 'Why Vienna Does So Well on Quality-of-Life Rankings #affordablehousing #publictransport #sustainability https://t.co/xLQASUUqoc',
 'RT @APLShipping: APL achieved a 50.7% reduction in CO2 emissions per transported container per kilometre in 2017, as compared to 2009. Read…',
 "RT @AmazonWatch: “Hydroelectricity may appear relatively 'clean' but the process in which a mega-dam is built is far from it... #BeloMonte…"]

**Count Vectorizer**

In [91]:
#Creating a count vectorizor over the first five tweets for 1-grams and 2-grams

vectorizer = CountVectorizer(stop_words = 'english',ngram_range=(1,2))
vectorizer.fit(five_tweets)

print(vectorizer.get_feature_names())

x = vectorizer.transform(five_tweets)

['2009', '2009 read', '2017', '2017 compared', '50', '50 reduction', 'achieved', 'achieved 50', 'affordablehousing', 'affordablehousing publictransport', 'amazonwatch', 'amazonwatch hydroelectricity', 'angelurena', 'angelurena visit', 'apl', 'apl achieved', 'aplshipping', 'aplshipping apl', 'appear', 'appear relatively', 'belomonte', 'bg7ewm1fco', 'billclinton', 'billclinton promote', 'built', 'built far', 'caribbean', 'caribbean president', 'clean', 'clean process', 'co2', 'co2 emissions', 'compared', 'compared 2009', 'concludes', 'concludes week', 'container', 'container kilometre', 'dam', 'dam built', 'does', 'does quality', 'ehs', 'ehs riskmanagement', 'emissions', 'emissions transported', 'far', 'far belomonte', 'https', 'https bg7ewm1fco', 'https xlqasuuqoc', 'hydroelectricity', 'hydroelectricity appear', 'join', 'join spfaustralia', 'kilometre', 'kilometre 2017', 'latest', 'latest trends', 'learn', 'learn latest', 'life', 'life rankings', 'mega', 'mega dam', 'melbourne', 'melbou

In [92]:
#Show the sparse vs. dense matrix

print('Sparse Matrix')
print(type(x))
print(x)

print ('Matrix')
x_back = x.toarray()
print(type(x_back))
print(x_back)

Sparse Matrix
<class 'scipy.sparse.csr.csr_matrix'>
  (0, 12)	1
  (0, 13)	1
  (0, 22)	1
  (0, 23)	1
  (0, 26)	1
  (0, 27)	1
  (0, 34)	1
  (0, 35)	1
  (0, 67)	1
  (0, 68)	1
  (0, 69)	1
  (0, 70)	1
  (0, 73)	1
  (0, 74)	1
  (0, 88)	1
  (0, 90)	1
  (0, 94)	1
  (0, 95)	1
  (0, 96)	1
  (0, 106)	1
  (0, 107)	1
  (0, 108)	1
  (0, 109)	1
  (1, 21)	1
  (1, 42)	1
  :	:
  (3, 100)	1
  (3, 101)	1
  (4, 10)	1
  (4, 11)	1
  (4, 18)	1
  (4, 19)	1
  (4, 20)	1
  (4, 24)	1
  (4, 25)	1
  (4, 28)	1
  (4, 29)	1
  (4, 38)	1
  (4, 39)	1
  (4, 46)	1
  (4, 47)	1
  (4, 51)	1
  (4, 52)	1
  (4, 63)	1
  (4, 64)	1
  (4, 71)	1
  (4, 72)	1
  (4, 84)	1
  (4, 85)	1
  (4, 88)	1
  (4, 89)	1
Matrix
<class 'numpy.ndarray'>
[[0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 1 1 0 0 0 0 0 0 1 1
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0
  0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1
  1 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0


In [93]:
#Creates pandas dataframe for all 111 words / word combinations (2) found in first five tweets

pd.DataFrame(x_back, columns=vectorizer.get_feature_names())

,2009,2009 read,2017,2017 compared,50,50 reduction,achieved,achieved 50,affordablehousing,affordablehousing publictransport,...,transported container,trends,trends ehs,vienna,vienna does,visit,visit concludes,week,week stops,xlqasuuqoc
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,1,...,0,0,0,1,1,0,0,0,0,1
3,1,1,1,1,1,1,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
#Words with a frequency higher than 1

pd.DataFrame(x_back, columns=vectorizer.get_feature_names()).sum()[pd.DataFrame(x_back, columns=vectorizer.get_feature_names()).sum() > 1]

https             2
rt                3
sustainability    2
dtype: int64

In [ ]:
#for doc in sustainability_collection.find({}):
#    print (doc['text'])

**TFIDF**

In [122]:
tfidf_vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2))
tweet_vectors = tfidf_vectorizer.fit_transform(five_tweets)

classes = np.array(['obj'] + ['sub'] + ['obj']*2+ ['sub'])

model = MultinomialNB().fit(tweet_vectors, classes)

In [204]:
for tweet in five_tweets:
    print(tweet)

RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…
Are you in Melbourne or Sydney? Join us at #SPFAustralia to learn about the latest trends in #EHS, #RiskManagement… https://t.co/BG7eWM1FCO
Why Vienna Does So Well on Quality-of-Life Rankings #affordablehousing #publictransport #sustainability https://t.co/xLQASUUqoc
RT @APLShipping: APL achieved a 50.7% reduction in CO2 emissions per transported container per kilometre in 2017, as compared to 2009. Read…
RT @AmazonWatch: “Hydroelectricity may appear relatively 'clean' but the process in which a mega-dam is built is far from it... #BeloMonte…


In [120]:
for tweet in five_tweets:
    print(TextBlob(tweet).sentiment)

Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.5, subjectivity=0.9)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.23333333333333334, subjectivity=0.8500000000000001)


In [188]:
#Pull sample of 10 tweets

cursor = sustainability_collection.aggregate([{'$sample':{'size':10}}])

In [189]:
curslist = list(cursor)

In [197]:
for doc in curslist:
    print ('Tweet: \n\n', doc['text'],'\n')

Tweet: 

 RT @Damien_CABADI: Should your country get hydrogen trains? 🤔

WORLD ECONOMIC FORUM

Congrats #Germany! #tech4good #sustainability #ecology… 

Tweet: 

 RT @ThinkroomTalk: Entrepreneurship | ANDE Says SMEs Need Ease Of Doing Business Policy
https://t.co/Ob2Vp4KZM5
@ANDEWestAfrica #Sustainabi… 

Tweet: 

 #GreenHashes #GH #Green #Mining #ICO #airdrop 

Tweet: 

 🔥🌐 RETWEET SHARE SUPPORT! 🌐🔥 @ScottHa55665467 Crisis in Nicaragua #SOSNicaragua #Sustainability @ViralExpNotify… https://t.co/S0FBkXXhoJ 

Tweet: 

 RT @AgnesAltmets: Had a brilliant time talking about sustainability &amp; sharing @AutoTrader_UK EV insight at the Manchester R event, organise… 

Tweet: 

 RT Capital_FinServ "RT Damien_CABADI: Should your country get hydrogen trains? 🤔

WORLD ECONOMIC FORUM

Congrats… https://t.co/goROmTKS1z 

Tweet: 

 Just booked my ticket to @GRSBeef‘s #GlobalBeef18 conference – looking forward to plenty of discussion on cattle… https://t.co/27ISwCv7tT 

Tweet: 

 RT @Rohan_Patole88: 

In [209]:
for tweet in curslist:
    
    print(TextBlob(tweet['text']).sentiment)

Sentiment(polarity=0.25, subjectivity=0.2)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=-0.2, subjectivity=0.3)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.9, subjectivity=1.0)
Sentiment(polarity=0.2, subjectivity=0.2)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.6, subjectivity=0.5166666666666667)
Sentiment(polarity=0.0, subjectivity=0.0)
Sentiment(polarity=0.0, subjectivity=0.0)


In [304]:
sustainability_collection.find_one()

{'_id': ObjectId('5b70e2403386b77533324b26'),
 'created_at': 'Mon Aug 13 01:41:17 +0000 2018',
 'id': 1028818744703950849,
 'id_str': '1028818744703950849',
 'text': 'RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'angelurena',
    'name': 'Angel Ureña',
    'id': 70707457,
    'id_str': '70707457',
    'indices': [3, 14]},
   {'screen_name': 'BillClinton',
    'name': 'Bill Clinton',
    'id': 1330457336,
    'id_str': '1330457336',
    'indices': [95, 107]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_nam

**Review Sample of 100 Tweets**

In [317]:
#Pull sample of 100 tweets

cursor_100 = sustainability_collection.aggregate([{'$sample':{'size':100}}])

tweets_100 = list(cursor_100)

In [383]:
'''

Review sample of 100 tweets:

    User Name
    Location of User
    Text
    Retweet (Number of retweets of the original tweet)
    Sentiment Analysis of Text
    
'''
rows_list = []

for tweet in tweets_100:

        tweet_dict = {}

        tweet_dict['user'] = tweet['user']['name']
        tweet_dict['screenname'] = tweet['user']['screen_name']
        tweet_dict['location'] = tweet['user']['location']
        tweet_dict['text'] = tweet['text']
        tweet_dict['retweet'] = tweet['retweet_count']
        tweet_dict['num_followers']= tweet['user']['followers_count']
        tweet_dict['sentiment'] = TextBlob(tweet['text']).sentiment

        rows_list.append(tweet_dict)

tweet_df = pd.DataFrame(rows_list)

In [384]:
tweet_df

,location,num_followers,retweet,screenname,sentiment,text,user
0,South Africa,500867,0,RichSimmondsZA,"(-0.1, 0.05)",RT @evankirstel: Factory reset: Tech startups ...,Rich Simmonds
1,Melbourne Australia,1071,0,Super_Koala_,"(-0.2, 0.0)",Help us to keep recycling recyclable. Otherwis...,Super Koala
2,"Kamloops, BC",5258,0,TRUsustain,"(0.0, 0.0)",Check out this tree-mendous 🌳 skyscraper in To...,TRU Sustainability
3,"Atlanta, GA USA",12966,0,EngineersDay,"(0.0, 0.0)",[Job] District Representative II | Company: Ec...,Engineering Daily
4,NYC 📍Lived in 🇨🇳🇫🇷🇬🇧🇺🇸,1600,0,DanniZMeunier,"(0.2333333333333333, 0.43333333333333335)",RT @berrinmarsh: A gorgeous royal blue wool cr...,INNOVATION+
5,"Québec, Canada",1346,0,ElmoSolutions,"(0.5, 0.5)",#RT SiemensPLM #Shipping regulations have push...,Elmo Solutions
6,"Oakland, CA",3523,0,WhitishJ,"(0.2, 0.3)",@ShaneCompany 2/2 We care about sustainability...,Jacob Whitish
7,"London, England",42,0,PostTagStories,"(-0.07777777777777779, 0.6444444444444445)",Could #electricvehicles be the key to unlockin...,PostTag
8,None,4,0,Jason_Sunstone,"(-0.2, 0.3)",#GreenHashes #GH #Green #Mining #ICO #airdrop,Z.Sen
9,,14,33,hultgrenanders,"(0.0, 0.0)",RT @sust_insight: Climate change: 'Hothouse Ea...,Anders Hultgren


In [381]:
tweet_df[tweet_df['retweet']>0]

,location,retweet,screenname,sentiment,text,user
9,,33,hultgrenanders,"(0.0, 0.0)",RT @sust_insight: Climate change: 'Hothouse Ea...,Anders Hultgren
33,"Chennai, India",5,deepak007raj,"(0.0, 0.0)",RT @nasir27: Sustainability! https://t.co/YOx0...,Deepak Raj
34,"Queensland, Australia",1,QLD_RFA,"(0.13636363636363635, 0.45454545454545453)",RT @rsb_org: Did you hear about our new partne...,QRFA
37,,24,JairoEzequiel,"(-0.4, 0.7)",RT @Marangus77: Argentina: While my debt susta...,Jairo!
67,London,2,BatterseaNet,"(0.5, 0.5)",Top story: ELLE UK Dedicates September Issue T...,Battersea Internet
80,Singapore,2,GreenEventsAsia,"(0.45, 0.6)",RT @sumanthc: #CarbonCoach : shining a light o...,Sustainable Events A
81,"Sydney, Oz",3,OzCazza64,"(0.0, 0.0)",RT @ConservationOrg: We’ve teamed up with @Lif...,Carole Crawford
89,,1,X8936551024,"(0.6, 1.0)",@HungryComrade Why don't we mandate subsistenc...,_________
98,,69,dzhray,"(1.0, 0.9)",RT @jblefevre60: No need a pilot’s license to ...,Ray


In [337]:
sustainability_collection.find_one()

{'_id': ObjectId('5b70e2403386b77533324b26'),
 'created_at': 'Mon Aug 13 01:41:17 +0000 2018',
 'id': 1028818744703950849,
 'id_str': '1028818744703950849',
 'text': 'RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'angelurena',
    'name': 'Angel Ureña',
    'id': 70707457,
    'id_str': '70707457',
    'indices': [3, 14]},
   {'screen_name': 'BillClinton',
    'name': 'Bill Clinton',
    'id': 1330457336,
    'id_str': '1330457336',
    'indices': [95, 107]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_nam

In [338]:
sustainability_collection.find_one()['user']['followers_count']

790

In [ ]:
['retweeted_status']['id']

**20 Most Popular Hashtags**

In [210]:
sustainability_collection.count()

11073

In [214]:
hashtag_cursor = sustainability_collection.aggregate([
    {'$match': {'entities.hashtags.0':{'$exists':1}}},
    {'$project': {'_id':0, 'hashtags':'$entities.hashtags'}},
    {'$unwind': '$hashtags'},
    {'$group': {'_id': '$hashtags.text', 'count': {'$sum':1}}},
    {'$sort': {'count':-1}},
    {'$limit':20}
])

In [215]:
list(hashtag_cursor)

[{'_id': 'sustainability', 'count': 1894},
 {'_id': 'Sustainability', 'count': 1816},
 {'_id': 'plastic', 'count': 318},
 {'_id': 'climatechange', 'count': 283},
 {'_id': 'IoT', 'count': 263},
 {'_id': 'environment', 'count': 229},
 {'_id': 'waste', 'count': 219},
 {'_id': 'energy', 'count': 212},
 {'_id': 'Green', 'count': 208},
 {'_id': 'innovation', 'count': 179},
 {'_id': 'drones', 'count': 166},
 {'_id': 'SDGs', 'count': 162},
 {'_id': 'tech', 'count': 152},
 {'_id': 'SmartCity', 'count': 148},
 {'_id': 'job', 'count': 146},
 {'_id': 'ICO', 'count': 143},
 {'_id': 'Industry40', 'count': 142},
 {'_id': 'airdrop', 'count': 141},
 {'_id': 'BigData', 'count': 141},
 {'_id': 'Mining', 'count': 141}]

In [597]:
hashtag_cursor = sustainability_collection.aggregate([
    {'$match': {'entities.hashtags.0':{'$exists':1}}},
    {'$project': {'_id':0, 'hashtags':'$entities.hashtags'}},
    {'$unwind': '$hashtags'},
    {'$group': {'_id': '$hashtags.text', 'count': {'$sum':1}}},
    {'$sort': {'count':-1}},
    {'$limit':20}
])

In [598]:
hashtag_list = []
for i in list(hashtag_cursor):
    hashtag_list.append(i) 

In [599]:
hashtag_list

[{'_id': 'sustainability', 'count': 1894},
 {'_id': 'Sustainability', 'count': 1816},
 {'_id': 'plastic', 'count': 318},
 {'_id': 'climatechange', 'count': 283},
 {'_id': 'IoT', 'count': 263},
 {'_id': 'environment', 'count': 229},
 {'_id': 'waste', 'count': 219},
 {'_id': 'energy', 'count': 212},
 {'_id': 'Green', 'count': 208},
 {'_id': 'innovation', 'count': 179},
 {'_id': 'drones', 'count': 166},
 {'_id': 'SDGs', 'count': 162},
 {'_id': 'tech', 'count': 152},
 {'_id': 'SmartCity', 'count': 148},
 {'_id': 'job', 'count': 146},
 {'_id': 'ICO', 'count': 143},
 {'_id': 'Industry40', 'count': 142},
 {'_id': 'airdrop', 'count': 141},
 {'_id': 'BigData', 'count': 141},
 {'_id': 'Mining', 'count': 141}]

In [594]:
pd.DataFrame(hashtag_list).rename(columns={"_id": "Hashtags", "count":"Count"}) 

,Hashtags,Count
0,sustainability,1894
1,Sustainability,1816
2,plastic,318
3,climatechange,283
4,IoT,263
5,environment,229
6,waste,219
7,energy,212
8,Green,208
9,innovation,179


In [371]:
sustainability_hashtags = []
for hashtag in hashtag_list:   
    sustainability_hashtags.append(hashtag['_id'])

In [372]:
sustainability_hashtags

['sustainability',
 'Sustainability',
 'plastic',
 'climatechange',
 'IoT',
 'environment',
 'waste',
 'energy',
 'Green',
 'innovation',
 'drones',
 'SDGs',
 'tech',
 'SmartCity',
 'job',
 'ICO',
 'Industry40',
 'airdrop',
 'BigData',
 'Mining']

In [600]:
with open ('hashtags.pickle', 'wb') as to_write:
    pickle.dump(sustainability_hashtags, to_write)

In [340]:
sustainability_collection.find_one()

{'_id': ObjectId('5b70e2403386b77533324b26'),
 'created_at': 'Mon Aug 13 01:41:17 +0000 2018',
 'id': 1028818744703950849,
 'id_str': '1028818744703950849',
 'text': 'RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'angelurena',
    'name': 'Angel Ureña',
    'id': 70707457,
    'id_str': '70707457',
    'indices': [3, 14]},
   {'screen_name': 'BillClinton',
    'name': 'Bill Clinton',
    'id': 1330457336,
    'id_str': '1330457336',
    'indices': [95, 107]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_nam

**Top Followers**

In [353]:
followers_cursor = sustainability_collection.aggregate([
    {'$match': {'followers_count':{'$exists':1}}},
    {'$project': {'_id':0, 'user.screen_name','followers_count'}},
    {'$sort': {'followers_count'}},
    {'$limit':20}
])

SyntaxError: invalid syntax (<ipython-input-353-809d232d580f>, line 3)

In [438]:

new_cursor = sustainability_collection.aggregate([
 {'$project': {'_id':0, 'user.screen_name','followers_count'}},
    {'$sort': {'retweet_count':-1}},
    {'$limit':20}])


In [439]:
list(new_cursor)

[{'_id': ObjectId('5b70e3a73386b775333250f7'),
  'created_at': 'Sun Aug 12 17:40:13 +0000 2018',
  'id': 1028697679923224578,
  'id_str': '1028697679923224578',
  'text': 'RT @GreenHashes: Want to know why in 2018 Crypto will see massive shift in awareness?\nMathematical Race of Mining limited to Elite &amp; Techie…',
  'truncated': False,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'GreenHashes',
     'name': 'GreenHashes',
     'id': 947559842218065920,
     'id_str': '947559842218065920',
     'indices': [3, 15]}],
   'urls': []},
  'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
  'source': '<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'user': {'id': 1107555098,
   'id_str': '1107555098',
   'name': 'Héctor Varela Rey',
   'scree

**Check Retweets**

In [585]:
#Lists Top 20 Retweets

retweet_cursor = sustainability_collection.aggregate([
   #{'$project': {'_id':0, 'username':'$user.screen_name', 'text':'$text', 'retweets':'$retweet_count', 'status':'$retweeted_status.id'}},
    
    {'$project': {'text':1,'_id':'$retweeted_status.id' }},
  #  {'$sort': {'retweets':-1}},

    {'$group': {'_id':'$retweeted_status.id'}},#:{'$first'}}},# 'count': {'$sum':1}
    {'$limit':20}
])

In [586]:
retweets = list(retweet_cursor)

In [587]:
retweets

[{'_id': None}]

In [536]:
#Prints list of top 20 tweets to review for retweet count, usernames, topics

for tweet in retweets:
    print ('Tweet:\n',tweet['text'],'\n(Retweeted', tweet['retweet_count'],'Times)\n')

KeyError: 'text'

In [520]:
sustainability_collection.find_one({})

{'_id': ObjectId('5b70e2403386b77533324b26'),
 'created_at': 'Mon Aug 13 01:41:17 +0000 2018',
 'id': 1028818744703950849,
 'id_str': '1028818744703950849',
 'text': 'RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…',
 'truncated': False,
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'angelurena',
    'name': 'Angel Ureña',
    'id': 70707457,
    'id_str': '70707457',
    'indices': [3, 14]},
   {'screen_name': 'BillClinton',
    'name': 'Bill Clinton',
    'id': 1330457336,
    'id_str': '1330457336',
    'indices': [95, 107]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_nam

**Filter on tweets in English only**

In [291]:
english_cursor = sustainability_collection.aggregate([
    {'$match': {'lang': 'en'}},
])

In [292]:
english_list = list(english_cursor)

In [356]:
for doc in english_list[:5]:
    print(doc['text'])

RT @angelurena: This visit concludes this week’s stops in Miami and the Caribbean by President @BillClinton to promote sustainability and r…
Are you in Melbourne or Sydney? Join us at #SPFAustralia to learn about the latest trends in #EHS, #RiskManagement… https://t.co/BG7eWM1FCO
Why Vienna Does So Well on Quality-of-Life Rankings #affordablehousing #publictransport #sustainability https://t.co/xLQASUUqoc
RT @APLShipping: APL achieved a 50.7% reduction in CO2 emissions per transported container per kilometre in 2017, as compared to 2009. Read…
RT @AmazonWatch: “Hydroelectricity may appear relatively 'clean' but the process in which a mega-dam is built is far from it... #BeloMonte…


In [294]:
len(english_list)

10474

In [296]:
import pickle

In [297]:
with open ('sustainability_english.pickle', 'wb') as to_write:
    pickle.dump(english_list, to_write)